JSON structure-- {"text": "[INST] <<SYS>>\nFor the given two sentences, classify them as semantically similar with 'yes' or 'no'\n<</SYS>>\n\nSentence 1: A plane is taking off.\nSentence 2: An air plane is taking off.\nAre they semantically similar?:\n[/INST]Response:yes"}

In [122]:
import json
import re
import string
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

In [2]:
import pandas as pd

In [22]:
column_names = ['captions', 'meta-data1', 'meta-data2', 'meta-data3', 'similarity_score', 'Sentence1', 'Sentence2']

In [99]:
sts_train_json = pd.read_json('Dataset/sts/sts-train.json',lines=True)
sts_test_json = pd.read_json('Dataset/sts/sts-test.json',lines=True)
sts_train = pd.read_csv('Dataset/sts/sts-train.csv', sep="\t", header=None, names=column_names)
sts_test = pd.read_csv('Dataset/sts/sts-test.csv', sep="\t", header=None, names=column_names)
msr_train_json = pd.read_json('Dataset/msr_latest/msr-train.json',lines=True)
msr_test_json = pd.read_json('Dataset/msr_latest/msr-test.json',lines=True)
qqp_train_json = pd.read_json('Dataset/qqp/qqp-train.json',lines=True)
qqp_test_json = pd.read_json('Dataset/qqp/qqp-test.json',lines=True)

In [25]:
type(sts_train_json)

pandas.core.frame.DataFrame

In [26]:
sts_train_json.head()

,text
0,"[INST] <<SYS>>\nFor the given two sentences, c..."
1,"[INST] <<SYS>>\nFor the given two sentences, c..."
2,"[INST] <<SYS>>\nFor the given two sentences, c..."
3,"[INST] <<SYS>>\nFor the given two sentences, c..."
4,"[INST] <<SYS>>\nFor the given two sentences, c..."


In [103]:
def preprocessing(passed_json):
    pattern = r'\w+|[^\w\s]+'
    total_common_count=0
    total_noncommon_count=0
    sentence1_array=[]
    sentence2_array=[]
    target_output=[]
    for i in passed_json["text"]:
        S1= re.search(r"Sentence 1:(.*)",i).group(1)
        S2= re.search(r"Sentence 2:(.*)",i).group(1)
        response = re.search(r"Response:(.*)",i).group(1)
        sentence1_array.append(S1)
        sentence2_array.append(S2)
        target_output.append(1 if response == "yes" else 0)
        #punctuations = [char for char in S1 if char in string.punctuation]
       # print(punctuations)
        #punct.extend(punctuations)
        #S1 = re.sub()
        s1_tokens = re.findall(pattern, S1)
        s2_tokens = re.findall(pattern, S2)

        S1.lower()
        S2.lower()
        s1_tokens = re.findall(pattern, S1)
        s2_tokens = re.findall(pattern, S2)

        counter1 = Counter(s1_tokens)
        counter2 = Counter(s2_tokens)

        # Find common elements and their counts
        common_elements_count = (counter1 & counter2).items()
        unique_to_list1 = counter1 - counter2
        unique_to_list2 = counter2 - counter1

        # Combine the results to mimic the symmetric difference
        unique_elements = unique_to_list1 + unique_to_list2

        non_common = sum(unique_elements.values())
        total_noncommon_count += non_common
        # Calculate the total number of common elements
        total_common = sum(min(counter1[element], counter2[element]) for element in (counter1 & counter2))
        total_common_count += total_common
    return sentence1_array,sentence2_array, target_output, total_common_count, total_noncommon_count

In [104]:
sts_train_s1, sts_train_s2, sts_target_output_train, sts_commoncount_train, sts_noncommoncount_train=preprocessing(sts_train_json)
sts_test_s1, sts_test_s2, sts_target_output_test, sts_commoncount_test, sts_noncommoncount_test=preprocessing(sts_test_json)
msr_train_s1, msr_train_s2, msr_target_output_train, msr_commoncount_train, msr_noncommoncount_train=preprocessing(msr_train_json)
msr_test_s1, msr_test_s2, msr_target_output_test, msr_commoncount_test, msr_noncommoncount_test=preprocessing(msr_test_json)
qqp_train_s1, qqp_train_s2, qqp_target_output_train, qqp_commoncount_train, qqp_noncommoncount_train=preprocessing(qqp_train_json)
qqp_test_s1, qqp_test_s2, qqp_target_output_test, qqp_commoncount_test, qqp_noncommoncount_test=preprocessing(qqp_test_json)

Common and non common words in STS

Common and non common words in MSR

Common and non common words in QQP


In [105]:
print(sts_commoncount_train+sts_commoncount_test, sts_noncommoncount_train+sts_noncommoncount_test)
print(msr_commoncount_train+msr_commoncount_test, msr_noncommoncount_train+msr_noncommoncount_test)
print(qqp_commoncount_train+qqp_commoncount_test, qqp_noncommoncount_train+qqp_noncommoncount_test)

46440 72319
88363 86436
29967 68888


In [129]:
def get_sim_tfidf(s1,s2):
    # Combine the sentences into one list for vectorization
    all_sentences = s1 + s2

    # Initialize a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the sentences
    tfidf_matrix = vectorizer.fit_transform(all_sentences)

    # Calculate cosine similarity
    #cos_sim = cosine_similarity([vec_sentence1], [vec_sentence2])[0][0]

    # Define a threshold
    threshold = 0.5
    # Generate embeddings
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(all_sentences)

    # Now, each pair of sentences at the same index can be compared
    similarities_tf = []
    similarities_sbert = []
    for i in range(len(s1)):
        # Compute cosine similarity between corresponding sentence pairs for TF-IDF
        sim_score_tf = cosine_similarity(tfidf_matrix[i], tfidf_matrix[len(s1) + i])[0][0]
        
        # Compute cosine similarity between corresponding sentence pairs for SBERT
        sim_score_sbert = cosine_similarity(
            embeddings[i].reshape(1, -1),  # Reshape embeddings to 2D
            embeddings[len(s1) + i].reshape(1, -1)  # Reshape embeddings to 2D
        )[0][0]

        # Append the binarized similarity scores
        similarities_tf.append(1 if sim_score_tf >= threshold else 0)
        similarities_sbert.append(1 if sim_score_sbert >= threshold else 0)

    return similarities_tf, similarities_sbert

In [134]:
sts_train_simtf,sts_train_simsbert= get_sim_tfidf(sts_train_s1,sts_train_s2)
sts_test_simtf, sts_test_simsbert= get_sim_tfidf(sts_test_s1,sts_test_s2)
msr_train_simtf, msr_train_simsbert= get_sim_tfidf(msr_train_s1,msr_train_s2)
msr_test_simtf, msr_test_simsbert= get_sim_tfidf(msr_test_s1,msr_test_s2)
qqp_train_simtf, qqp_train_simsbert= get_sim_tfidf(qqp_train_s1,qqp_train_s2)
qqp_test_simtf, qqp_test_simsbert= get_sim_tfidf(qqp_test_s1,qqp_test_s1)

In [152]:
def get_scores(target_output, similarities, title):

    conf_matrix = confusion_matrix(target_output, similarities)
    #print("Confusion Matrix:")
    #print(conf_matrix)
    #disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['No', 'Yes'])
    #disp.plot(cmap=plt.cm.Blues)
    #plt.title(title)
    #plt.show()

    # Calculate precision, recall, and F1-score
    precision = precision_score(target_output, similarities)
    recall = recall_score(target_output, similarities)
    f1 = f1_score(target_output, similarities)
    acc_score = accuracy_score(target_output, similarities)

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Accuracy Score: {acc_score:.2f}")
    print("#####")

Let's see how 3 datasets vary in similarity(cosine), when the tfidf/Sbert is fitted for only train data. 
1) First, let's check accuracy of similarity for the train data of the three datasets (SBERT representation)
2) check accuracy of similarity for the test data of the three datasets (SBERT representation)

3) check accuracy of similarity for the train data of the three datasets (tf-idf representation) 
4) check accuracy of similarity for the test data of the three datasets (tf-idf representation) 

In [153]:
get_scores(sts_target_output_train,sts_train_simsbert, "Consufion matrix of train data -- SBERT -- STS")
#get_scores(sts_target_output_test,sts_test_simsbert)

get_scores(msr_target_output_train,msr_train_simsbert, "Consufion matrix of train data -- SBERT -- MSR")
#get_scores(msr_target_output_test,msr_test_simsbert)

get_scores(qqp_target_output_train,qqp_train_simsbert, "Consufion matrix of train data -- SBERT -- QQP")
#get_scores(qqp_target_output_test,qqp_test_simsbert)

Precision: 0.67
Recall: 0.98
F1 Score: 0.80
Accuracy Score: 0.74
#####
Precision: 0.70
Recall: 0.99
F1 Score: 0.82
Accuracy Score: 0.70
#####
Precision: 0.49
Recall: 1.00
F1 Score: 0.65
Accuracy Score: 0.60
#####


### TEST DATA - SBERT

In [154]:

get_scores(sts_target_output_test,sts_test_simsbert, "Confusion matrix of test data -- SBERT -- STS")

get_scores(msr_target_output_test,msr_test_simsbert, "Confusion matrix of test data -- SBERT -- MSR")

get_scores(qqp_target_output_test,qqp_test_simsbert, "Confusion matrix of test data -- SBERT -- QQP")

Precision: 0.68
Recall: 0.96
F1 Score: 0.80
Accuracy Score: 0.76
#####
Precision: 0.68
Recall: 0.99
F1 Score: 0.81
Accuracy Score: 0.69
#####
Precision: 0.35
Recall: 1.00
F1 Score: 0.51
Accuracy Score: 0.35
#####


### TRAIN - TFIDF REPRESENTATION

In [155]:
# test set
get_scores(sts_target_output_train,sts_train_simtf, "Consufion matrix of train data -- TF-IDF -- STS")
#get_scores(sts_target_output_test,sts_test_simsbert)

get_scores(msr_target_output_train,msr_train_simtf, "Consufion matrix of train data -- TF-IDF -- MSR")
#get_scores(msr_target_output_test,msr_test_simsbert)

get_scores(qqp_target_output_train,qqp_train_simtf, "Consufion matrix of train data -- TF-IDF -- QQP")


Precision: 0.78
Recall: 0.74
F1 Score: 0.76
Accuracy Score: 0.75
#####
Precision: 0.74
Recall: 0.89
F1 Score: 0.81
Accuracy Score: 0.71
#####
Precision: 0.54
Recall: 0.71
F1 Score: 0.61
Accuracy Score: 0.66
#####


In [156]:
# test set
get_scores(sts_target_output_test,sts_test_simtf, "Consufion matrix of test data -- TF-IDF -- STS")
#get_scores(sts_target_output_test,sts_test_simsbert)

get_scores(msr_target_output_test,msr_test_simtf, "Consufion matrix of test data -- TF-IDF -- MSR")
#get_scores(msr_target_output_test,msr_test_simsbert)

get_scores(qqp_target_output_test,qqp_test_simtf, "Consufion matrix of test data -- TF-IDF -- QQP")


Precision: 0.76
Recall: 0.72
F1 Score: 0.74
Accuracy Score: 0.75
#####
Precision: 0.73
Recall: 0.90
F1 Score: 0.81
Accuracy Score: 0.71
#####
Precision: 0.35
Recall: 1.00
F1 Score: 0.51
Accuracy Score: 0.35
#####
